In [1]:
# Loading Dependencies
import cv2 as cv
import numpy as np
print(np.__version__)
import tensorflow as tf
print(tf.__file__)
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers
from keras import ops

import os
data_directory = "../data/raw/"

1.26.4
/Users/lucas/Developer/captcha-solver/env/lib/python3.9/site-packages/tensorflow/__init__.py
TensorFlow version: 2.16.2
Num GPUs Available: 1


In [2]:
# Loading data
images = []
images_files = [i for i in os.listdir(data_directory)]
print(images_files[0:5])
print(len(images_files))

['32Q4.png', '97SX.png', '9K8W.png', 'WV2B.png', '5M8F.png']
9955


In [3]:
# Processign one image
processed_folder = "../data/processed/"
counts = {}
for image in images_files:
    img_directory = data_directory + image
    captcha = image[0:4]
    img = cv.imread(img_directory, cv.IMREAD_GRAYSCALE)
    img = cv.copyMakeBorder(img, 10, 10, 10, 10, cv.BORDER_REPLICATE)
    inverted = cv.bitwise_not(img)
    _, threshold_image = cv.threshold(inverted, 0, 255, cv.THRESH_BINARY)
    contours, hierarchy = cv.findContours(threshold_image.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    bounding_boxes = [cv.boundingRect(i) for i in contours]
    contours_sorted = sorted(bounding_boxes, key = lambda x: x[0])

    letter_image_regions = []

    for cnt in contours_sorted:
        x, y, w, h = cnt
        # cv.rectangle(img_coloured, (x,y), (x+w, y+h), (0, 255, 0), 1)
        if w/h > 1.25:
            half_width = int(w/2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))
        else:
            letter_image_regions.append((x, y, w, h))
    
    if len(letter_image_regions) != 4:
        continue
    
    for letter_contour_box, letter in zip(letter_image_regions, captcha):
        x, y, w, h = letter_contour_box
        roi = img[y:y+h, x:x+w]

        output_path = os.path.join(processed_folder, letter)

        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        count = counts.get(letter, 1)
        image_path = os.path.join(output_path, f'{str(count).zfill(6)}.png')
        
        cv.imwrite(image_path, roi)

        counts[letter] = count + 1


In [3]:
model = keras.Sequential([
    layers.Convolution2D(32, (3,3), padding = "same", input_shape=(28,28,1), activation="relu"),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Convolution2D(64, (3, 3), padding = "same", activation="relu"),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(32, activation="softmax"),
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

/Users/lucas/Developer/captcha-solver/env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-20 20:04:18.273302: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2025-07-20 20:04:18.273564: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-07-20 20:04:18.273582: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-07-20 20:04:18.274035: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-20 20:04:18.274055: I tensorflow/core/common_runtime/pluggable_device/pluggabl

In [4]:
train_data = tf.keras.utils.image_dataset_from_directory(
    "../data/processed",
    validation_split = 0.2,
    subset = "training",
    seed = 69,
    labels = "inferred",
    label_mode = "categorical",
    color_mode = "grayscale",
    image_size = (28,28),
    batch_size = 16
)

test_data = tf.keras.utils.image_dataset_from_directory(
    "../data/processed",
    validation_split = 0.2,
    subset = "validation",
    seed = 69,
    labels = "inferred",
    label_mode = "categorical",
    color_mode = "grayscale",
    image_size = (28,28),
    batch_size = 16
)

normalization = keras.layers.Rescaling(1./255)
train_data = train_data.map(lambda x, y: (normalization(x), y))
test_data = test_data.map(lambda x, y: (normalization(x), y))

Found 38752 files belonging to 32 classes.
Using 31002 files for training.
Found 38752 files belonging to 32 classes.
Using 7750 files for validation.


In [5]:
train_data = train_data.prefetch(tf.data.AUTOTUNE)
test_data = test_data.prefetch(tf.data.AUTOTUNE)
history = model.fit(
    train_data,
    validation_data = test_data,
    epochs = 10
)

Epoch 1/10


: 